# __Regression task__

#### Libraries and session's settings

In [1]:
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMRegressor

from hyper_tuner.core import make_hyperopt
from hyper_tuner.metrics import MAPE

import config

In [2]:
N_SAMPLES = 100000
N_FEATURES = 20
N_INFORMATIVE = 10

#### Generate dataset

In [3]:
X, y = make_regression(n_samples=N_SAMPLES,
                       n_features=N_FEATURES,
                       n_informative=N_INFORMATIVE,
                       random_state=42)
X = pd.DataFrame(X, columns=[f"x{i}" for i in range(1,N_FEATURES+1)])
y = pd.Series(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

#### Simple Model

In [4]:
model = LGBMRegressor().fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"MAPE: {100*MAPE(y_test, y_pred):.1f}%")

MAPE: 61.6%


#### Make hyper tuning

In [5]:
tuner_pipe = make_hyperopt(estimator=config.ESTIMATOR,
                           X=X, y=y,
                           cv=config.CV,
                           space=config.SPACE, common_params=config.COMMON_PARAMS,
                           fit_params=config.FIT_PARAMS, predict_params=config.PREDICT_PARAMS,
                           cat_params=[], int_params=config.INT_PARAMS,
                           max_evals=2)

100%|██████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.25s/trial, best loss: 14.657120473611053]


In [ ]:
model = LGBMRegressor(**tuner_pipe.best_parameters).fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"MAPE: {100*MAPE(y_test, y_pred):.1f}%")